In [1]:
# Math stuff
import matplotlib.pyplot as plt 
import numpy as np
import random 
import math
import pandas as pd

# Data stuff
import csv
import os
import sys
import json


/home/zhongxuan/ync/compgeom/robustness_test/data/


# Mathematical Functions

In [2]:
def coeffGen(p,k):
    """
    Input: Lower limit and higher limit (int)
    Output: List of length K with random values between low and high
    """
    coeffSeq = []
    for i in range(k):
        # mean = a, var = b^2
        coeffSeq.append(np.random.normal(0, 1.0/np.power(i+1, p)))
    return coeffSeq

def genTrigFun(a_k, b_k):
    """
    Input: 
    >> int list a_k, b_k: Sequences of length K
    
    Output: 
    >> fun f(t): Desired trig function with coefficients corresponding to sequences a_k, b_k
    """
    def fun_t (t):
        sum = 0
        for i in range(len(a_k)):
            sum += a_k[i]*np.sin(i*t) + b_k[i]*np.cos(i*t)
        return sum
    return (fun_t)

def euclidDist(a, b):
    return np.sqrt(np.power(a, 2) + np.power(b, 2))

def speed(x_t, y_t):
    """
    Input: 2 arrays of length of TIME_T
    Output: 1 array of length of TIME_T representing the speed 
    """
    return euclidDist(np.gradient(x_t), np.gradient(y_t))
    
def curvature(x_t, y_t):
    """
    Input: 2 arrays of length of TIME_T
    Output: 1 array of length of TIME_T representing the curvature
    """    
    num = abs(np.gradient(x_t)*np.gradient(np.gradient(y_t))
             - np.gradient(y_t)*np.gradient(np.gradient(x_t)))
    denom = np.power(speed(x_t, y_t), 3)
    return (num/denom)

def changeResolution (num, res):
    if num == 0 or res == 0 or math.isnan(num) or math.isnan(res):
        return 0
    return math.floor(num * res) / res

# Curve Generating Functions

In [3]:
# Define constants
NUM_CURVES = 50
NUM_TERMS = 50
POWER_OF_TERMS = 2.1
SIZE = 100

In [12]:
def genCurve(k, p, size, file_no):
    """
    Inputs: 
    >> int k: Number of terms in trig series 
    >> int p: Power of denominator when sampled from normal distribution
    >> int s: size - boundary box size
    >> int file_no: Index, used for naming files
    
    Outputs:
    >> Dataframe with columns [a_k, b_k, c_k, d_k], coefficients used to generate parametric curve
    
    Writes the data to Curves/coefficients_(file_no) in a csv file.
    """
    
    # Coefficients - 1 x K
    a_k = coeffGen(p, k)
    b_k = coeffGen(p, k)
    c_k = coeffGen(p, k)
    d_k = coeffGen(p, k)
    
    # Theta
    theta = random.random()**0.5 * (2*math.pi)
    
    x_min = size/2.0*random.random()**2
    x_max = size - size/2.0*random.random()**2
    
    y_min = size/2.0*random.random()**2
    y_max = size - size/2.0*random.random()**2
    
    # Transform data into dataframe
    data = np.transpose(np.array((a_k, b_k, c_k, d_k)))
    df1 = pd.DataFrame(data, columns = ['a_k', 'b_k', 'c_k', 'd_k'])
    extras = np.transpose(np.array([theta, size, x_min, x_max, y_min, y_max]))
    df2 = pd.DataFrame(extras, columns = ['extras'])
    df = pd.concat([df1,df2], axis=1)
    
    # Export csv
    curve_file_csv = 'curves/coefficients_' + str(file_no) + '.csv'
    df.to_csv(curve_file_csv, index=False)
    return df

In [13]:
def genNCurves(n, k, p, s):
    """
    Inputs: 
    >> int n: Number of curves we want to generate
    >> int k: Number of terms in trig series 
    >> int p: Power of denominator when sampled from normal distribution
    
    Outputs nothing.
    Writes n csv files with desired curve data.
    """
    for i in range(n):
        # Uniformly sample k from a range of 2 to 100 to get a bigger variety of curves
        k = random.randrange(2,100)
        # Use grid size 100x100
        genCurve(k, p, 100, i)
    
def captureCurve(filename_prefix, file_no, time_start, time_stop, frame_rate, resolution, plot=True):
    """
    Inputs: 
    >> int file_no:                   File number (corresponds to the file Curves/coefficients_(file_no).csv)
    >> float time_start, time_stop:   Beginning and end times to generate time step increments
    >> int frame_rate:                How often to sample the time step between time_start and time_stop
    >> string file_out:               Name of output file 
    >> bool plot:                     Plots the captured curves if set to true
    
    Outputs:
    >> dataframe df:                  Dataframe with columns [X, Y, SPEED, CURVATURE, (TODO) ARCLENGTH] 
    >> dataframe summaryStats:        Dataframe with columns for each summary statistics
    
    Writes 2 csv files with curve data and summary stats as above.
    """    

    # Read in data from coefficients csv
    # Coefficients - 1 x K
    data = pd.read_csv("curves/coefficients_" + str(file_no) + ".csv")
    
    a_k = data['a_k'].values
    b_k = data['b_k'].values
    c_k = data['c_k'].values
    d_k = data['d_k'].values
    
    theta = data['extras'][0]
    
    size = data['extras'][1]
    
    x_min = data['extras'][2]
    x_max = data['extras'][3]
    x_dif = x_max - x_min
    
    y_min = data['extras'][4]
    y_max = data['extras'][5]
    y_dif = y_max - y_min
    
    
    time_t = np.arange(time_start, time_stop, 1/frame_rate)
    p_time_t = theta * (time_t - time_start) / (time_stop - time_start)

    x_fun = genTrigFun(a_k, b_k)
    y_fun = genTrigFun(c_k, d_k)
    
    # Get minimum and maximum x,y coordinates of the graph 
    
    x_og = x_fun(p_time_t) # 1 x len(TIME_T)
    y_og = y_fun(p_time_t) # 1 x len(TIME_T)
    
    lower_xlim = min(x_og)
    upper_xlim = max(x_og)
    
    lower_ylim = min(y_og)
    upper_ylim = max(y_og)
    
    # Coordinates - 1 x len(TIME_T)
    
    x_enlarged = []
    y_enlarged = []
    for i in range(0, len(time_t)):
        x_enlarged.append((x_dif / (upper_xlim - lower_xlim) * (x_og[i] - lower_xlim) + x_min))
        y_enlarged.append((y_dif / (upper_ylim - lower_ylim) * (y_og[i] - lower_ylim) + y_min))
    
    # Transform coordinates based on resolution
    
    x = []
    y = []
    for i in range(0, len(time_t)):
        x.append(np.array(changeResolution(x_enlarged[i], resolution)))
        y.append(np.array(changeResolution(y_enlarged[i], resolution)))

    # Speed and curvature - 1 x len(TIME_T)
    spd = speed(x, y) * frame_rate
    crv = curvature(x, y)
    
    # Calculate arc length starting from second point
    arclen = np.zeros(len(time_t))
    for i in range(1, len(time_t)):
        x1 = x[i-1]
        y1 = y[i-1]
        x2 = x[i]
        y2 = y[i]
        increment = euclidDist((x2-x1), (y2-y1))
        arclen[i] = arclen[i-1] + increment
    
    if(plot):
        # Plots
        plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)
        plt.subplot(231)
        plt.plot(x, y)
        plt.title("Curve no. " + str(file_no) + ", m = " + str(len(a_k)))
        plt.axis([0,size,0,size])

        plt.subplot(232)
        kSeq = np.arange(0, len(a_k), 1)
        plt.plot(kSeq, a_k)
        plt.plot(kSeq, b_k)
        plt.plot(kSeq, c_k)
        plt.plot(kSeq, d_k)
        plt.title("Coefficients")
        plt.axis([0,20,-1,1])

        plt.subplot(233)
        plt.plot(time_t, spd)
        plt.title("Speed")
        plt.axis([time_start,time_stop,0,3*frame_rate])

        plt.subplot(234)
        plt.plot(time_t, crv)
        plt.title("Curvature")
        plt.axis([time_start,time_stop,0,20])
        
        plt.subplot(235)
        plt.plot(time_t, arclen)
        plt.title("Arc Length")
        plt.axis([time_start,time_stop,0,3*size])
        
        plt.show()

    # Transform data into dataframe
    data = np.transpose(np.array((x, y)))
    df = pd.DataFrame(data, columns = ['X', 'Y'])
    
    # Export csv
    if filename_prefix == "FR_TEST_":
        data_file_csv = 'curve_testing/data/frame_rate_tests_data/' + filename_prefix + 'CRV_' + str(file_no) + '_FR_' + str(frame_rate) +  '_RES_' + str(resolution) + '.dat'
        df.to_csv(data_file_csv)
    elif filename_prefix == "RES_TEST_":
        data_file_csv = 'curve_testing/data/resolution_tests_data/' + filename_prefix + 'CRV_' + str(file_no) + '_FR_' + str(frame_rate) +  '_RES_' + str(resolution) + '.dat'
        df.to_csv(data_file_csv)
    
    summaryStats = df.describe()
    summary_file_csv = 'curve_testing/summary_stats/' + filename_prefix + 'CRV_' + str(file_no) + '_FR_' + str(frame_rate) +  '_RES_' + str(resolution) + '.dat'
    summaryStats.to_csv(summary_file_csv)
    return[df, summaryStats]

- Capture N curves is used to capture many curves with the same frame rate and resolution

In [14]:
def captureNCurves(n, time_start, time_stop, frame_rate, resolution, bool_plot):
    """
    Inputs:
    >> int n: Number of curves to capture
    ** Check genNCurves for the rest.
    
    Outputs nothing.
    Write n csv files with curve data and n csv files with curve summary statistics.
    Writes a json files with all the n curves and their file locations.
    """
    jsonItems = []
    for i in range(n):
        file_name = "CRV_" + str(i) + "_FR_" + str(frame_rate) +  "_RES_" + str(resolution)
        captureCurve("", i, time_start, time_stop, frame_rate, resolution, plot = bool_plot)
        jsonItem = {
            "name": file_name,
            "data_file_location": PATH_TO_DIRECTORY + "/curve_testing/data/" + file_name +".dat",
            "animal_attributes":
            {
              "species": "Magic Scoliosis Fish",
              "exp_type": "MCS funtimes",
              "ID": str(i),
              "control_group": "False"
            },
            "capture_attributes": 
            {
              "dim_x": 100,
              "dim_y": 100,
              "pixels_per_mm": resolution,
              "frames_per_sec": frame_rate,
              "start_time": 0,
              "end_time": 10,
              "baseline_start_time": 0,
              "baseline_end_time": 10
            }
          }
        jsonItems.append(jsonItem)
    outfilename = PATH_TO_DIRECTORY + "/robustness_test.json"
    jsonstr = json.dumps(jsonItems, indent=4)
    with open(outfilename, "w") as outfile:
        outfile.write(jsonstr)
    print("Wrote the information into %s" % outfilename)

In [15]:
genNCurves(NUM_CURVES, NUM_TERMS, POWER_OF_TERMS, SIZE)

# Capture n curves - Testing
               #n  start  stop  frame rate resolution  plot
# captureNCurves(10, 0,     5,    60,        20,        False)

# Robustness Testing

- captureTestFrCurves and captureTestResCurves captures one curve repeatedly, fixing one variable and changing the other

In [17]:
# Define constants
DEFAULT_FR = 24
DEFAULT_RES = 2
DEFAULT_START = 0
DEFAULT_STOP = 5

TEST_FR = [6, 12, 24, 48, 96] # Frames per second
TEST_RES = [0.5, 1, 2, 4, 8] # Pixels per mm

def captureTestFrCurves(file_no):   
    """
    Inputs:
    >> int n: Number of curves to capture
    ** Check genNCurves for the rest.
    
    Outputs nothing.
    Write n csv files with curve data and n csv files with curve summary statistics.
    Writes a json files with all the n curves and their file locations.
    """
    jsonItems = []
    n = len(TEST_FR)
    for i in range(n):
        frame_rate = TEST_FR[i]
        resolution = DEFAULT_RES
        df = captureCurve("FR_TEST_", file_no, DEFAULT_START, DEFAULT_STOP, frame_rate, resolution, plot=False)[0][['X', 'Y']]        
        
        file_name = "FR_TEST_CRV_" + str(file_no) + "_FR_" + str(frame_rate) +  "_RES_" + str(resolution)
        
        jsonItem = {
            "name": file_name,
            "data_file_location": PATH_TO_DIRECTORY + "/curve_testing/data/frame_rate_tests_data/" + file_name +".dat",
            "animal_attributes":
            {
              "species": "Magic Scoliosis Fish",
              "exp_type": "MCS funtimes",
              "ID": str(i),
              "control_group": "False"
            },
            "capture_attributes": 
            {
              "dim_x": 100,
              "dim_y": 100,
              "pixels_per_mm": resolution,
              "frames_per_sec": frame_rate,
              "start_time": 0,
              "end_time": 10,
              "baseline_start_time": 0,
              "baseline_end_time": 10
            }
          }
        jsonItems.append(jsonItem)
    outfilename = PATH_TO_DIRECTORY + "/frame_rate_tests/CRV_" + str(file_no) + '.json'
    jsonstr = json.dumps(jsonItems, indent=4)
    with open(outfilename, "w") as outfile:
        outfile.write(jsonstr)
    print("Wrote the information into %s" % outfilename)

def captureTestResCurves(file_no):
    jsonItems = []
    n = len(TEST_RES)
    for i in range(5):
        frame_rate = DEFAULT_FR
        resolution = TEST_RES[i]
        df = captureCurve("RES_TEST_", file_no, DEFAULT_START, DEFAULT_STOP, frame_rate, resolution, plot=False)[0][['X', 'Y']]

        file_name = "RES_TEST_CRV_" + str(file_no) + "_FR_" + str(frame_rate) +  "_RES_" + str(resolution)
        
        jsonItem = {
            "name": file_name,
            "data_file_location": PATH_TO_DIRECTORY + "/curve_testing/data/resolution_tests_data/" + file_name +".dat",
            "animal_attributes":
            {
              "species": "Magic Scoliosis Fish",
              "exp_type": "MCS funtimes",
              "ID": str(i),
              "control_group": "False"
            },
            "capture_attributes": 
            {
              "dim_x": 100,
              "dim_y": 100,
              "pixels_per_mm": resolution,
              "frames_per_sec": frame_rate,
              "start_time": 0,
              "end_time": 10,
              "baseline_start_time": 0,
              "baseline_end_time": 10
            }
          }
        jsonItems.append(jsonItem)
    outfilename = PATH_TO_DIRECTORY + "/resolution_tests/CRV_" + str(file_no) + '.json'
    jsonstr = json.dumps(jsonItems, indent=4)
    with open(outfilename, "w") as outfile:
        outfile.write(jsonstr)
    print("Wrote the information into %s" % outfilename)

for i in range(NUM_CURVES):
    captureTestFrCurves(i)
    captureTestResCurves(i)

/home/zhongxuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide


Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CRV_0.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_0.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CRV_1.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_1.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CRV_2.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_2.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CRV_3.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_3.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CRV_4.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_4.json


Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_41.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CRV_42.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_42.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CRV_43.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_43.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CRV_44.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_44.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CRV_45.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/resolution_tests/CRV_45.json
Wrote the information into /home/zhongxuan/ync/compgeom/robustness_test/frame_rate_tests/CR

# Checking Diversity of Curve Library

In [20]:
# DEPRECATED - capture curve no longer stores curvature, speed and arc length

def captureCurveCSV(file_no, time_start, time_stop, frame_rate, resolution, bool_plot):
    df_curv = pd.DataFrame() 
    for i in range(NUM_CURVES):
        df_curv[i] = captureCurve(i, time_start, time_stop, frame_rate, resolution, plot = bool_plot)[0]['CURVATURE']
    # Export to CSV
    df_curv.to_csv('curve_diversity_stats/' + str(file_no) + 'curvature.csv')

def captureSpeedCSV(file_no, time_start, time_stop, frame_rate, resolution, bool_plot):
    df_curv = pd.DataFrame() 
    for i in range(NUM_CURVES):
        df_curv[i] = captureCurve(i, time_start, time_stop, frame_rate, resolution, plot = bool_plot)[0]['SPEED']
    # Export to CSV
    df_curv.to_csv('curve_diversity_stats/' + str(file_no) + 'speed.csv')
    
def captureArcCSV(file_no, time_start, time_stop, frame_rate, resolution, bool_plot):
    df_curv = pd.DataFrame() 
    for i in range(NUM_CURVES):
        df_curv[i] = captureCurve(i, time_start, time_stop, frame_rate, resolution, plot = bool_plot)[0]['ARCLENGTH']
    # Export to CSV
    df_curv.to_csv('curve_diversity_stats/' + str(file_no) + 'arclength.csv')
    
# captureCurveCSV(10,0,5,60, 20, False)
# captureSpeedCSV(10,0,5,60, 20, False)
# captureArcCSV(10,0,5,60, 20, False)

# Generate Summary Statistics across N curves for each variable

In [21]:
# DEPRECATED - capture curve no longer stores curvature, speed and arc length

N =10

def varStats (var_str):
    """
    Inputs: 
    >> string var_str: Variable (X, Y, SPEED, CURVATURE or ARCLENGTH) that we want to check summary statistics on
    
    Outputs:
    >> dataframe df: Dataframe with columns of summary statistics for that variable
    """
    df = pd.DataFrame(columns = ["count", "mean", "std", "min", "25%", "50%", "75%", "max"])
    for i in range(N):
        curve_i = pd.read_csv("Curve Data/summary_stats_" + str(i) + ".csv")
        row = pd.DataFrame(curve_i[var_str]).T
        row.columns = ["count", "mean", "std", "min", "25%", "50%", "75%", "max"]
        df = df.append(row)
    return df

# allVarStats = pd.DataFrame(columns = ["count", "mean", "std", "min", "25%", "50%", "75%", "max"])
# for df in (varStats('X'), varStats('Y'), varStats('SPEED'), varStats('CURVATURE'). varStats('ARCLENGTH')):
#     allVarStats = allVarStats.append(df)
    
# allVarStats.to_csv('summary_stats_' + str(N) + '_curves.csv')
# XStats = varStats('X')
# YStats = varStats('Y')
# spdStats = varStats('SPEED')
# crvStats = varStats('CURVATURE')
# arcStats = varStats('ARCLENGTH')